In [2]:
import psycopg2
from scipy.stats import pearsonr
from postgres_config import POSTGRES_DB, POSTGRES_HOST, POSTGRES_PASSWORD, POSTGRES_PORT, POSTGRES_USER
conn = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    database=POSTGRES_DB 
)
cursor = conn.cursor()

# query view of DISTINCT call types
cursor.execute("SELECT typ_desc FROM sch_nypd_calls_tables.types_of_calls")
typ_desc_values = cursor.fetchall()

# empty list to store correlation values
correlations = []

# iterate through all the call types against the daily high temp
for typ_desc in typ_desc_values:
    query = """
    SELECT
        COUNT(CASE WHEN typ_desc = %s THEN 1 END) as count,
        twd.daily_temp_maximum
    FROM
        sch_nypd_calls_tables.tb_call_data tcd
    INNER JOIN
        sch_nypd_calls_tables.tb_weather_data twd
    ON DATE(tcd.incident_date) = twd.DATE
    GROUP BY
        date(incident_date), daily_temp_maximum
    """

    # this will esxcute them each and return the result
    cursor.execute(query, (typ_desc[0],))
    result = cursor.fetchall()
    
    # create two lists for count and temperature
    counts, temps = zip(*result)
    
    # calculate the correlation coefficient for each pairing
    corr, _ = pearsonr(counts, temps)
    
    # then we add them to our correlation list
    correlations.append((typ_desc[0], corr))

# close the database connection
cursor.close()
conn.close()

# Sthen we rank the typ_desc values based on the correlation coefficients
correlations.sort(key=lambda x: x[1], reverse=True)  # Sort by the correlation coefficient

# finally we output the ranking onto the juptyter console
print("Ranked typ_desc by correlation coefficient:")
for typ_desc, corr in correlations:
    print(f"{typ_desc}: {corr}")


Ranked typ_desc by correlation coefficient:
AMBULANCE CASE: UNCONSCIOUS/OUTSIDE: 0.7908609339593504
DISORDERLY: PERSON/OUTSIDE: 0.7523249549529346
AMBULANCE CASE: EDP/OUTSIDE: 0.6733410215539636
OTHER CRIMES (IN PROGRESS): HARASSMENT/OUTSIDE: 0.6159444794765049
OTHER CRIMES (IN PROGRESS): TRESPASS/OUTSIDE: 0.5887622188809819
INVESTIGATE/POSSIBLE CRIME: CALLS FOR HELP/OUTSIDE: 0.5585306962558197
INVESTIGATE/POSSIBLE CRIME: FIREARM/OUTSIDE: 0.5393076907385098
INVESTIGATE/POSSIBLE CRIME: NARCO SALES/OUTSIDE: 0.5248097330053896
DISPUTE: KNIFE/OUTSIDE: 0.4970330209495084
LARCENY (PAST): OTHER/OUTSIDE: 0.4924988020240833
ASSAULT (IN PROGRESS): OTHER/OUTSIDE: 0.48413742980893065
ASSAULT (PAST): OTHER/OUTSIDE: 0.48105248057194055
LARCENY (IN PROGRESS): OTHER/OUTSIDE: 0.47327072986357144
ASSAULT (IN PROGRESS): WEAPON/OUTSIDE: 0.47047020488523045
ASSAULT (IN PROGRESS): KNIFE/OUTSIDE: 0.46926788021573373
OTHER CRIMES (PAST): HARASSMENT/OUTSIDE: 0.46769558707086994
INVESTIGATE/POSSIBLE CRIME: KNIF

## Ranked Correlation of 911 Calls vs. S&P 500 CLosing Price OBSOLETE

In [11]:
import psycopg2
from scipy.stats import pearsonr
from A_postgres_config import POSTGRES_DB, POSTGRES_HOST, POSTGRES_PASSWORD, POSTGRES_PORT, POSTGRES_USER

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    database=POSTGRES_DB 
)
cursor = conn.cursor()

# Retrieve unique typ_desc values
cursor.execute("SELECT typ_desc FROM sch_nypd_calls_tables.types_of_calls")
typ_desc_values = cursor.fetchall()

# This will store all counts and percent_changes for each typ_desc
data = {}

for typ_desc in typ_desc_values:
    # Query to fetch all percent_changes for each typ_desc
    query = """
    SELECT
        COUNT(*) as count,
        ts5.percent_change
    FROM 
        sch_nypd_calls_tables.tb_call_data tcb
    RIGHT JOIN
        (
            SELECT
                date,
                (close - LAG(close, 252) OVER (ORDER BY date)) / LAG(close, 252) OVER (ORDER BY date) * 100 AS percent_change
            FROM 
                sch_nypd_calls_tables.tb_sp_500
        ) ts5
    ON 
        DATE(tcb.incident_date) = ts5.date
    WHERE tcb.typ_desc = %s
    GROUP BY ts5.date, ts5.percent_change
    """

    # Execute the query for the current typ_desc
    cursor.execute(query, (typ_desc[0],))
    results = cursor.fetchall()

    # Store results in the data dictionary
    if typ_desc[0] not in data:
        data[typ_desc[0]] = {'counts': [], 'percent_changes': []}
    
    for count, percent_change in results:
        if percent_change is not None:
            data[typ_desc[0]]['counts'].append(count)
            data[typ_desc[0]]['percent_changes'].append(percent_change)

# Close the database connection
cursor.close()
conn.close()

# Now calculate correlations
correlations = []
for typ_desc, values in data.items():
    if len(values['counts']) > 1 and len(values['percent_changes']) > 1:
        corr, _ = pearsonr(values['counts'], values['percent_changes'])
        correlations.append((typ_desc, corr))

# Rank the typ_desc values based on the correlation coefficients
correlations.sort(key=lambda x: x[1], reverse=True)  # Sort by the correlation coefficient

# Output the ranked list
print("Ranked typ_desc by correlation coefficient:")
for typ_desc, corr in correlations:
    print(f"{typ_desc}: {corr}")


KeyboardInterrupt: 

In [10]:
import psycopg2
from A_postgres_config import POSTGRES_DB, POSTGRES_HOST, POSTGRES_PASSWORD, POSTGRES_PORT, POSTGRES_USER

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    database=POSTGRES_DB 
)
cursor = conn.cursor()

# Query distinct call types
cursor.execute("SELECT typ_desc FROM sch_nypd_calls_tables.types_of_calls")
typ_desc_values = cursor.fetchall()

# Iterate through all the call types
for typ_desc_tuple in typ_desc_values:
    typ_desc = typ_desc_tuple[0]  # Extract the string value from the tuple
    query = f"""
    WITH Counts AS (
        SELECT
            boro_nm,
            COUNT(CASE WHEN typ_desc = '{typ_desc}' THEN 1 END) AS count
        FROM
            sch_nypd_calls_tables.tb_call_data
        GROUP BY
            boro_nm
        HAVING 
            COUNT(CASE WHEN typ_desc = '{typ_desc}' THEN 1 END) >= 1
    ),
    Ranked AS (
        SELECT
            boro_nm,
            count,
            RANK() OVER(ORDER BY count DESC) AS rank
        FROM
            Counts
    )
    SELECT
        '{typ_desc}' AS typ_desc,
        count,
        rank
    FROM
        Ranked
    WHERE
        (rank = 1 OR rank = 2) AND boro_nm = 'STATEN ISLAND';
    """
    cursor.execute(query)
    results = cursor.fetchall()
    for result in results:
        print(f"Type: {result[0]}, Count: {result[1]}, Rank: {result[2]}")


Type: FIRE: BRUSH/LTD ACC HWY, Count: 56, Rank: 2
Type: BURGLARY (PAST): OTHER/SCHOOL, Count: 9, Rank: 2
Type: LARCENY (IN PROGRESS): OTHER/SCHOOL, Count: 4, Rank: 2
Type: OTHER CRIMES (IN PROGRESS): VIOL ORDER PROTECT/LTD ACC HWY, Count: 1, Rank: 2
Type: AMBULANCE CASE: BURN SERIOUS/LTD ACC HWY, Count: 2, Rank: 1
Type: ASSAULT (IN PROGRESS): SHOTS/SCHOOL, Count: 1, Rank: 2
Type: LARCENY (PAST): VEHICLE/TRANSIT, Count: 2, Rank: 1
Type: OTHER CRIMES (IN PROGRESS): TRESPASS/LTD ACC HWY, Count: 2, Rank: 1
Type: ASSAULT (PAST): SHOTS/CHILD ABUSE, Count: 2, Rank: 1
Type: DISORDERLY: NOISE/LTD ACC HWY, Count: 1, Rank: 2
